### Experiment week 35: Baseline

In [2]:
from agatha.construct.semrep_handler import SemRepHandler
import json
import os
import pandas as pd
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from pathlib import Path
from tqdm import tqdm
import openai
import networkx as nx
from openai_llm import oai_get_response
import textwrap
import re
from openai import OpenAI
from tqdm import tqdm
import pandas as pd

from BaseLine_Path_Exp import PathExplainer
import pandas as pd

In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [10]:
explainer = PathExplainer('/work/acslab/shared/Agatha_shared/sentences.sqlite3')

### 1. Read Labeled paths as positive_negative paths:

In [6]:
pos_neg_df = pd.read_pickle('pos_neg_eval_pred_w_paths_downsampled_df.pkl')
pos_neg_df

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031...","[C3658706, C0278579, C4724800]"
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['29500171', '28946...","[C3658706, C4704928, C0278579, C4724800]"
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['33983696', '34956...","[C3658706, C4704928, C4724800]"
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C1367202'): ['27764796', '28881...","[C3658706, C1367202, C4288736, C4724800]"
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2935436'): ['29881714', '34829...","[C3658706, C2935436, C4288736, C4724800]"
...,...,...,...,...,...,...,...,...,...,...
0,C0000970 C1332807,acetaminophen,CTSL gene,35745615,Analgesics Induce Alterations in the Expressio...,sr_pred_sent_5,2022,1,"{('C0000970', 'C1336641'): ['33375616', '26503...","[C0000970, C1336641, C1332807]"
0,C0000970 C1332807,acetaminophen,CTSL gene,35745615,Analgesics Induce Alterations in the Expressio...,sr_pred_sent_5,2022,1,"{('C0000970', 'C1422064'): ['34384355', '32655...","[C0000970, C1422064, C1332807]"
0,C0000970 C1332807,acetaminophen,CTSL gene,35745615,Analgesics Induce Alterations in the Expressio...,sr_pred_sent_5,2022,0,"{('C0000970', 'C1334098'): ['34080217', '30393...","[C0000970, C1334098, C0023810, C1332807]"
0,C0000970 C1332807,acetaminophen,CTSL gene,35745615,Analgesics Induce Alterations in the Expressio...,sr_pred_sent_5,2022,0,"{('C0000970', 'C0812222'): ['26011634', '16330...","[C0000970, C0812222, C1332807]"


### 2. select 10 pairs => 53 paths

In [7]:
unique_pairs = pos_neg_df['pair'].unique()[:10]
filtered_df = pos_neg_df[pos_neg_df['pair'].isin(unique_pairs)]
filtered_df

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031...","[C3658706, C0278579, C4724800]"
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['29500171', '28946...","[C3658706, C4704928, C0278579, C4724800]"
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['33983696', '34956...","[C3658706, C4704928, C4724800]"
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C1367202'): ['27764796', '28881...","[C3658706, C1367202, C4288736, C4724800]"
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2935436'): ['29881714', '34829...","[C3658706, C2935436, C4288736, C4724800]"
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2982078'): ['30859062'], ('C29...","[C3658706, C2982078, C4288736, C4724800]"
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C0265325'): ['26141621'], ('C02...","[C3467876, C0265325, C4524315]"
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C2697961'): ['32774277'], ('C26...","[C3467876, C2697961, C0265325, C4524315]"
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C1415138'): ['23432625'], ('C14...","[C3467876, C1415138, C0169658, C4524315]"
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C4519250'): ['33991955'], ('C45...","[C3467876, C4519250, C3853694, C4524315]"


In [8]:
len(filtered_df)

53

### 3. Apply baseline row wsie in df:

In [11]:
def generate_llm_response_bl(row):
    source = row['subj_name']
    target = row['obj_name']
    context_pmids = row['context_pmids']
    llm_response = explainer.explain_path(source, target, context_pmids)
    return llm_response

filtered_df['llm_response_bl'] = filtered_df.apply(generate_llm_response_bl, axis=1)
filtered_df

Retrieving abstracts: 100%|█████████████████████████████████████████████████| 2/2 [00:00<00:00, 25191.02it/s]


Time taken for generating response from llm(gpt4o): 5.02 seconds


Retrieving abstracts: 100%|█████████████████████████████████████████████████| 3/3 [00:00<00:00, 37786.52it/s]


Time taken for generating response from llm(gpt4o): 4.06 seconds


Retrieving abstracts: 100%|█████████████████████████████████████████████████| 2/2 [00:00<00:00, 24966.10it/s]


...

Retrieving abstracts: 100%|█████████████████████████████████████████████████| 3/3 [00:00<00:00, 27715.67it/s]


Time taken for generating response from llm(gpt4o): 7.70 seconds


Retrieving abstracts: 100%|█████████████████████████████████████████████████| 3/3 [00:00<00:00, 27473.61it/s]


Time taken for generating response from llm(gpt4o): 5.74 seconds


,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,llm_response_bl
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031...","[C3658706, C0278579, C4724800]",To describe an indirect relationship between p...
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['29500171', '28946...","[C3658706, C4704928, C0278579, C4724800]","Based on the provided abstract, an indirect re..."
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['33983696', '34956...","[C3658706, C4704928, C4724800]",An indirect relationship between pembrolizumab...
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C1367202'): ['27764796', '28881...","[C3658706, C1367202, C4288736, C4724800]",An indirect relationship between pembrolizumab...
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2935436'): ['29881714', '34829...","[C3658706, C2935436, C4288736, C4724800]",An indirect relationship between pembrolizumab...
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2982078'): ['30859062'], ('C29...","[C3658706, C2982078, C4288736, C4724800]",An indirect relationship between pembrolizumab...
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C0265325'): ['26141621'], ('C02...","[C3467876, C0265325, C4524315]",An indirect relationship between larotrectinib...
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C2697961'): ['32774277'], ('C26...","[C3467876, C2697961, C0265325, C4524315]",An indirect relationship between larotrectinib...
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C1415138'): ['23432625'], ('C14...","[C3467876, C1415138, C0169658, C4524315]",An indirect relationship between larotrectinib...
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C4519250'): ['33991955'], ('C45...","[C3467876, C4519250, C3853694, C4524315]",An indirect relationship between larotrectinib...


In [12]:
filtered_df

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,llm_response_bl
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031...","[C3658706, C0278579, C4724800]",To describe an indirect relationship between p...
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['29500171', '28946...","[C3658706, C4704928, C0278579, C4724800]","Based on the provided abstract, an indirect re..."
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['33983696', '34956...","[C3658706, C4704928, C4724800]",An indirect relationship between pembrolizumab...
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C1367202'): ['27764796', '28881...","[C3658706, C1367202, C4288736, C4724800]",An indirect relationship between pembrolizumab...
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2935436'): ['29881714', '34829...","[C3658706, C2935436, C4288736, C4724800]",An indirect relationship between pembrolizumab...
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2982078'): ['30859062'], ('C29...","[C3658706, C2982078, C4288736, C4724800]",An indirect relationship between pembrolizumab...
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C0265325'): ['26141621'], ('C02...","[C3467876, C0265325, C4524315]",An indirect relationship between larotrectinib...
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C2697961'): ['32774277'], ('C26...","[C3467876, C2697961, C0265325, C4524315]",An indirect relationship between larotrectinib...
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C1415138'): ['23432625'], ('C14...","[C3467876, C1415138, C0169658, C4524315]",An indirect relationship between larotrectinib...
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C4519250'): ['33991955'], ('C45...","[C3467876, C4519250, C3853694, C4524315]",An indirect relationship between larotrectinib...


### 4. Evaluate based on semrep for terms based jaccard index:

In [16]:
llm_model = "gpt-4o"
open('OAI_CONFIG_LIST').read().strip()
client = OpenAI(
    api_key=open('OAI_CONFIG_LIST').read().strip(),
)

def oai_get_response(msg, temp, top_p, seed=None):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": msg,
            }
        ],
        model=llm_model,
        temperature=temp,
        top_p=top_p,
    )
    
    reply_text = chat_completion.choices[0].message.content
    
    return reply_text

In [14]:
nlm_soft_folder = '/work/acslab/users/svalipou/SemRep'
sr_temp_folder = '/work/acslab/users/svalipou/semrep_20_aug'
sr_replace_utf8_path = '/work/acslab/users/svalipou/SemRep/replace_utf8.jar'

In [17]:
sr_h = SemRepHandler(
    nlm_soft_path=nlm_soft_folder,
    temp_folder=sr_temp_folder,
    replace_utf8_path=sr_replace_utf8_path,
)

In [18]:
sr_h.sr_binary_path = Path('/work/acslab/users/svalipou/SemRep/public_semrep/bin/semrep.v1.9_2021AB')

In [19]:
def process_semrep_output(semrep_output):
    all_dfs = []
    for sentence_id, sentence_data in tqdm(semrep_output.items(), desc="Processing SemRep Output"):
        df = pd.DataFrame(sentence_data['terms'])
        df['sentence_id'] = sentence_id
        all_dfs.append(df)
    combined_df = pd.concat(all_dfs, ignore_index=True)
    return combined_df

def jaccard_index(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    if union == 0:  
        return 0
    return intersection / union

In [20]:
def compute_jaccard_index(row):
    exp_bl = row['llm_response_bl']
    future_ref = row['pmid_text']
    
    exp_bl_sr_out = sr_h.ProcessList_parallel([exp_bl])
    df_bl_nodes = process_semrep_output(exp_bl_sr_out)
    nodes_bl = set(df_bl_nodes['pref_name'])

    future_ref_sr_out = sr_h.ProcessList_parallel([future_ref])
    df_ref_nodes = process_semrep_output(future_ref_sr_out)
    nodes_ref = set(df_ref_nodes['pref_name'])

    ji_score = jaccard_index(nodes_bl, nodes_ref)
    return ji_score

### 6. Apply jaccard-index row wise between future_ref & baseline explanation:

In [21]:
%%time
filtered_df['jaccard_index_score_bl'] = filtered_df.apply(compute_jaccard_index, axis=1)
filtered_df

Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...
Run SemRep in interactive mode...
Processing input with replace_utf8.jar uti

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,llm_response_bl,jaccard_index_score_bl
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031...","[C3658706, C0278579, C4724800]",To describe an indirect relationship between p...,0.042857
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['29500171', '28946...","[C3658706, C4704928, C0278579, C4724800]","Based on the provided abstract, an indirect re...",0.032258
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['33983696', '34956...","[C3658706, C4704928, C4724800]",An indirect relationship between pembrolizumab...,0.031250
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C1367202'): ['27764796', '28881...","[C3658706, C1367202, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029851
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2935436'): ['29881714', '34829...","[C3658706, C2935436, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.030769
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2982078'): ['30859062'], ('C29...","[C3658706, C2982078, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029412
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C0265325'): ['26141621'], ('C02...","[C3467876, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.028169
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C2697961'): ['32774277'], ('C26...","[C3467876, C2697961, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.041096
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C1415138'): ['23432625'], ('C14...","[C3467876, C1415138, C0169658, C4524315]",An indirect relationship between larotrectinib...,0.027778
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C4519250'): ['33991955'], ('C45...","[C3467876, C4519250, C3853694, C4524315]",An indirect relationship between larotrectinib...,0.028169


In [22]:
filtered_df

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,llm_response_bl,jaccard_index_score_bl
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031...","[C3658706, C0278579, C4724800]",To describe an indirect relationship between p...,0.042857
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['29500171', '28946...","[C3658706, C4704928, C0278579, C4724800]","Based on the provided abstract, an indirect re...",0.032258
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['33983696', '34956...","[C3658706, C4704928, C4724800]",An indirect relationship between pembrolizumab...,0.031250
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C1367202'): ['27764796', '28881...","[C3658706, C1367202, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029851
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2935436'): ['29881714', '34829...","[C3658706, C2935436, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.030769
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2982078'): ['30859062'], ('C29...","[C3658706, C2982078, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029412
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C0265325'): ['26141621'], ('C02...","[C3467876, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.028169
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C2697961'): ['32774277'], ('C26...","[C3467876, C2697961, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.041096
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C1415138'): ['23432625'], ('C14...","[C3467876, C1415138, C0169658, C4524315]",An indirect relationship between larotrectinib...,0.027778
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C4519250'): ['33991955'], ('C45...","[C3467876, C4519250, C3853694, C4524315]",An indirect relationship between larotrectinib...,0.028169


In [26]:
filtered_df.to_pickle('filtered_df.pkl')

In [39]:
filtered_df= pd.read_pickle('filtered_df_now.pkl')
filtered_df

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,llm_response_bl,jaccard_index_score_bl,EvalCorrectness_score_bl,EvalCorrectness_feedback_bl
0,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031...","[C3658706, C0278579, C4724800]",To describe an indirect relationship between p...,0.042857,3.0,The generated answer is relevant to the user q...
1,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['29500171', '28946...","[C3658706, C4704928, C0278579, C4724800]","Based on the provided abstract, an indirect re...",0.032258,1.0,The generated answer is not relevant to the us...
2,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['33983696', '34956...","[C3658706, C4704928, C4724800]",An indirect relationship between pembrolizumab...,0.031250,1.0,The generated answer is not relevant to the us...
3,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C1367202'): ['27764796', '28881...","[C3658706, C1367202, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029851,2.0,The generated answer attempts to establish an ...
4,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2935436'): ['29881714', '34829...","[C3658706, C2935436, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.030769,2.0,The generated answer provides a general overvi...
5,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2982078'): ['30859062'], ('C29...","[C3658706, C2982078, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029412,2.0,The generated answer is relevant to the user q...
6,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C0265325'): ['26141621'], ('C02...","[C3467876, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.028169,2.0,The generated answer is relevant to the user q...
7,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C2697961'): ['32774277'], ('C26...","[C3467876, C2697961, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.041096,4.0,The generated answer is relevant and mostly co...
8,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C1415138'): ['23432625'], ('C14...","[C3467876, C1415138, C0169658, C4524315]",An indirect relationship between larotrectinib...,0.027778,3.0,The generated answer is relevant and provides ...
9,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C4519250'): ['33991955'], ('C45...","[C3467876, C4519250, C3853694, C4524315]",An indirect relationship between larotrectinib...,0.028169,3.0,The generated answer is relevant and provides ...


In [40]:
len(filtered_df)

53

### 7. Evaluate based on space in MedCPT and Pubmed Cosine Similarity

In [41]:
pubmed_tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
pubmed_model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")

medcpt_tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Article-Encoder")
medcpt_model = AutoModel.from_pretrained("ncbi/MedCPT-Article-Encoder")

In [42]:
def compute_embedding(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

results = []

# Iterate over each row in the DataFrame
for index, row in filtered_df.iterrows():
    pmid_text = row['pmid_text']
    llm_response_bl = row['llm_response_bl']
    
    # Compute embeddings using PubMedBERT
    embedding_pubmed_text = compute_embedding(pubmed_model, pubmed_tokenizer, pmid_text)
    embedding_pubmed_response = compute_embedding(pubmed_model, pubmed_tokenizer, llm_response_bl)
    
    # Compute embeddings using MedCPT
    embedding_medcpt_text = compute_embedding(medcpt_model, medcpt_tokenizer, pmid_text)
    embedding_medcpt_response = compute_embedding(medcpt_model, medcpt_tokenizer, llm_response_bl)
    
    # Calculate cosine similarities
    score_pubmed = cosine_similarity([embedding_pubmed_text], [embedding_pubmed_response])[0][0]
    score_medcpt = cosine_similarity([embedding_medcpt_text], [embedding_medcpt_response])[0][0]
    
    # Append results
    results.append({
        'index': index,
        'score_pubmed': score_pubmed,
        'score_medcpt': score_medcpt
    })


results_df = pd.DataFrame(results)

filtered_df = filtered_df.merge(results_df, left_index=True, right_on='index')
filtered_df = filtered_df.drop(columns=['index'])
filtered_df

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,llm_response_bl,jaccard_index_score_bl,EvalCorrectness_score_bl,EvalCorrectness_feedback_bl,score_pubmed,score_medcpt
0,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031...","[C3658706, C0278579, C4724800]",To describe an indirect relationship between p...,0.042857,3.0,The generated answer is relevant to the user q...,0.985991,0.922633
1,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['29500171', '28946...","[C3658706, C4704928, C0278579, C4724800]","Based on the provided abstract, an indirect re...",0.032258,1.0,The generated answer is not relevant to the us...,0.983068,0.894738
2,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['33983696', '34956...","[C3658706, C4704928, C4724800]",An indirect relationship between pembrolizumab...,0.031250,1.0,The generated answer is not relevant to the us...,0.982368,0.879453
3,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C1367202'): ['27764796', '28881...","[C3658706, C1367202, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029851,2.0,The generated answer attempts to establish an ...,0.985832,0.901788
4,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2935436'): ['29881714', '34829...","[C3658706, C2935436, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.030769,2.0,The generated answer provides a general overvi...,0.980942,0.890056
5,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2982078'): ['30859062'], ('C29...","[C3658706, C2982078, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029412,2.0,The generated answer is relevant to the user q...,0.986268,0.903166
6,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C0265325'): ['26141621'], ('C02...","[C3467876, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.028169,2.0,The generated answer is relevant to the user q...,0.989873,0.908774
7,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C2697961'): ['32774277'], ('C26...","[C3467876, C2697961, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.041096,4.0,The generated answer is relevant and mostly co...,0.987409,0.907826
8,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C1415138'): ['23432625'], ('C14...","[C3467876, C1415138, C0169658, C4524315]",An indirect relationship between larotrectinib...,0.027778,3.0,The generated answer is relevant and provides ...,0.985120,0.887187
9,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C4519250'): ['33991955'], ('C45...","[C3467876, C4519250, C3853694, C4524315]",An indirect relationship between larotrectinib...,0.028169,3.0,The generated answer is relevant and provides ...,0.990895,0.882501


In [43]:
filtered_df.to_pickle('filtered_df_now.pkl')

### 7.1. statistically verify mean in each row

In [34]:
mean_0_ji = filtered_df[filtered_df['label'] == 0]['jaccard_index_score_bl'].mean()
mean_1_ji = filtered_df[filtered_df['label'] == 1]['jaccard_index_score_bl'].mean()

print(f"Mean jaccard_index_score_bl for label 0: {mean_0_ji}")
print(f"Mean jaccard_index_score_bl for label 1: {mean_1_ji}")

Mean jaccard_index_score_bl for label 0: 0.03238933419065904
Mean jaccard_index_score_bl for label 1: 0.036842945309938785


In [35]:
mean_0_ec = filtered_df[filtered_df['label'] == 0]['EvalCorrectness_score_bl'].mean()
mean_1_ec = filtered_df[filtered_df['label'] == 1]['EvalCorrectness_score_bl'].mean()

print(f"Mean EvalCorrectness_score_bl for label 0: {mean_0_ec}")
print(f"Mean EvalCorrectness_score_bl for label 1: {mean_1_ec}")

Mean EvalCorrectness_score_bl for label 0: 2.6666666666666665
Mean EvalCorrectness_score_bl for label 1: 2.9565217391304346


In [37]:
mean_0_pubmed = filtered_df[filtered_df['label'] == 0]['score_pubmed'].mean()
mean_1_pubmed = filtered_df[filtered_df['label'] == 1]['score_pubmed'].mean()

print(f"Mean score_pubmed for label 0: {mean_0_pubmed}")
print(f"Mean score_pubmed for label 1: {mean_1_pubmed}")

Mean score_pubmed for label 0: 0.9846981763839722
Mean score_pubmed for label 1: 0.9848819375038147


In [38]:
mean_0_medcpt = filtered_df[filtered_df['label'] == 0]['score_medcpt'].mean()
mean_1_medcpt = filtered_df[filtered_df['label'] == 1]['score_medcpt'].mean()

print(f"Mean score_medcpt for label 0: {mean_0_pubmed}")
print(f"Mean score_medcpt for label 1: {mean_1_pubmed}")

Mean score_medcpt for label 0: 0.9846981763839722
Mean score_medcpt for label 1: 0.9848819375038147


### Future_ref 20 pairs shared one in 
https://docs.google.com/spreadsheets/d/14IEM0sKo3L4AOhih9sdYLkMrxNWfW8sv/edit?gid=1131770176#gid=1131770176


In [11]:
fu_re_df = pd.read_csv('fu_ref_20_pairs.csv')
fu_re_df

,Unnamed: 0,pair,subj_cui,subj_name,subj_text,obj_cui,obj_name,obj_text,pmid,source,pair_text,year,future_reference
0,207,C0079419 C1332809,C0079419,TP53 gene,p53,C1332809,CUL1 gene,CUL1,36267727,title_cooc,TP53 gene; CUL1 gene,2022,Inhibitory effect of CUL1 on atherosclerosis t...
1,14,C0003009 C1417064,C0003009,angiotensin II,angiotensin II,C1417064,MC4R gene,MC4R,35476598,title_cooc,angiotensin II; MC4R gene,2022,Melanocortin MC4R receptor is required for ene...
2,286,C1122962 C4519167,C1122962,gefitinib,gefitinib,C4519167,tucatinib,tucatinib,37711177,sr_pred_sent_7,gefitinib; tucatinib,2023,Small-molecule inhibitors of kinases in breast...
3,341,C1412993 C4519250,C1412993,EML4 gene,EML4,C4519250,anlotinib,anlotinib,35642002,title_cooc,EML4 gene; anlotinib,2022,Role of chemokine-mediated angiogenesis in res...
4,58,C0011777 C3179470,C0011777,dexamethasone,dexamethasone,C3179470,remimazolam,remimazolam,36897701,title_cooc,dexamethasone; remimazolam,2023,Effect of low-dose propofol combined with dexa...
5,169,C0042790 C1413498,C0042790,Vision Disorders,visual dysfunction,C1413498,CLN6 gene,CLN6,35140099,title_cooc,Vision Disorders; CLN6 gene,2022,Higher order visual dysfunction and myoclonic-...
6,46,C0009140 C1418263,C0009140,CoASH,CoASH,C1418263,PANK1 gene,PanK,38054469,sr_pred_sent_5,CoASH; PANK1 gene,2023,Cholesterol-Mediated Coenzyme A Depletion in C...
7,372,C1453431 C3490348,C1453431,"BECN1 protein, human",Beclin1,C3490348,empagliflozin,EMPA,36030589,sr_pred_sent_8,"BECN1 protein, human; empagliflozin",2022,Novel cardioprotective mechanism for Empaglifl...
8,61,C0012091 C2609414,C0012091,diclofenac,Diclofenac,C2609414,Acute kidney injury,Acute Renal Injury,35883752,title_cooc,diclofenac; Acute kidney injury,2022,The Flavonoid Hesperidin Methyl Chalcone Targe...
9,69,C0013089 C1417955,C0013089,doxorubicin,doxorubicin,C1417955,OPA3 gene,OPA3-NFS1,36924813,title_cooc,doxorubicin; OPA3 gene,2023,Hydrogen sulfide alleviates mitochondrial dama...
